In [1]:
# Remove warnings.
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Import needed libraries.
from crewai import Agent, Task, Crew

In [6]:
# Define our LLM.
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAI

load_dotenv()
api_key = os.environ['GOOGLE_API_KEY']


llm = GoogleGenerativeAI(model="models/gemini-pro", google_api_key=api_key)


In [7]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You are responsible for planning documentation on the topic: {topic}. Collect relevant information to help the audience learn and make informed decisions. Your work lays the foundation for the Content Writer.",
    llm=llm,
    allow_delegation=False,
	verbose=True
)

In [8]:
planner

Agent(role=Content Planner, goal=Plan engaging and factually accurate content on {topic}, backstory=You are responsible for planning documentation on the topic: {topic}. Collect relevant information to help the audience learn and make informed decisions. Your work lays the foundation for the Content Writer.)

In [9]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate documentation on the topic: {topic}",
    backstory="You are tasked with writing new documentation on the topic: {topic}, based on the Content Planner's outline and context. Follow the main objectives and provide objective insights, acknowledging opinions where applicable.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [10]:
editor = Agent(
    role="Editor",
    goal="Edit a given documentation piece for clarity, grammar, and alignment with the brand's voice.",
    backstory="You are responsible for editing the documentation, ensuring it is clear, grammatically correct, and aligned with the brand's voice.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [15]:
from crewai_tools import SerperDevTool

search_tool = SerperDevTool()

In [16]:
plan = Task(
    description=(
        "1. Identify the latest trends, key players, and noteworthy news on {topic}.\n"
        "2. Define the target audience, considering their interests and pain points.\n"
        "3. Develop a detailed content outline including {sections}.\n"
        "4. Include SEO keywords and relevant data or sources.\n"
    ),
    expected_output="A comprehensive content plan with an outline, audience analysis, SEO keywords, and resources.",
    tools=[search_tool],
    agent=planner,
)

In [17]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling chapter on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Name sections/subtitles engagingly.\n"
        "4. Structure content with an engaging introduction, insightful body, and summarizing conclusion.\n"
        "5. Proofread for grammatical errors and brand voice alignment.\n"
    ),
    expected_output="A well-written documentation piece in markdown format, ready for publication. Each section should have 2-3 paragraphs.",
    tools=[search_tool],
    agent=writer,
)

In [18]:
edit = Task(
    description=(
        "Proofread the given documentation for grammatical errors and alignment with the brand's voice."
    ),
    expected_output="A well-edited documentation chapter in markdown format, ready for publication.",
    tools=[search_tool],
    agent=editor,
)

In [19]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [20]:
result = crew.kickoff(inputs={"topic": "Introduction to RAG application", "sections": "What is RAG, How to use RAG, and RAG vs. other tools"})

 [2024-07-22 18:32:35][DEBUG]: == Working Agent: Content Planner
 [2024-07-22 18:32:35][INFO]: == Starting Task: 1. Identify the latest trends, key players, and noteworthy news on Introduction to RAG application.
2. Define the target audience, considering their interests and pain points.
3. Develop a detailed content outline including What is RAG, How to use RAG, and RAG vs. other tools.
4. Include SEO keywords and relevant data or sources.



> Entering new CrewAgentExecutor chain...
Action: Search the internet
Action Input: {'search_query': 'What is RAG application?'} 


Search results: Title: What is Retrieval Augmented Generation (RAG)? - Databricks
Link: https://www.databricks.com/glossary/retrieval-augmented-generation-rag
Snippet: Retrieval augmented generation, or RAG, is an architectural approach that can improve the efficacy of large language model (LLM) applications by leveraging ...
---
Title: What Is Retrieval-Augmented Generation aka RAG | NVIDIA Blogs
Link: https://blogs

In [25]:
from IPython.display import Markdown
Markdown(result.raw)

# Introduction to Retrieval Augmented Generation (RAG)

## What is RAG?

Retrieval Augmented Generation (RAG) is a cutting-edge natural language processing (NLP) technique that combines retrieval-based and generative approaches to enhance the accuracy and coherence of AI-generated text. Unlike traditional generative models that solely rely on internal language models, RAG incorporates an external knowledge retrieval step to ground the generation process in real-world information. This hybrid approach enables RAG models to produce more factually correct, comprehensive, and contextually relevant text.

## How Does RAG Work?

RAG operates in two primary stages: retrieval and generation. In the retrieval stage, the model searches an external knowledge base, such as a large text corpus or database, for information relevant to the input text. This retrieved knowledge is then fed into a generative model, which uses it to inform and augment the generation process. By leveraging both retrieved knowledge and internal language patterns, RAG can produce text that is not only fluent and coherent but also grounded in factual evidence.

## Benefits of Using RAG

RAG offers several key advantages over traditional generative models:

- **Improved Factual Accuracy and Consistency**: By incorporating external knowledge, RAG models can generate text that is more factually correct and consistent with real-world information.
- **Enhanced Contextual Relevance**: RAG's retrieval-based approach ensures that the generated text is highly relevant to the input context, making it more useful and informative.
- **Increased Coherence and Fluency**: The combination of retrieval and generation enables RAG models to produce text that is both coherent and fluent, avoiding the disjointed or nonsensical output that can sometimes occur with purely generative models.

## Challenges of Using RAG

Despite its advantages, RAG also presents certain challenges:

- **Computational Cost**: The retrieval process in RAG can be computationally expensive, especially when working with large knowledge bases. This can limit the scalability and real-time applicability of RAG models.
- **Knowledge Base Dependency**: RAG's performance heavily relies on the quality and comprehensiveness of the external knowledge base used for retrieval. Biases or limitations in the knowledge base can propagate into the generated text.
- **Domain Specificity**: RAG models are typically trained on specific domains or tasks, limiting their applicability to other domains without additional fine-tuning.

## RAG vs. Other Generative AI Tools

RAG differs from other generative AI tools in several key ways:

- **Hybrid Approach**: RAG's unique combination of retrieval and generation sets it apart from purely generative models like GPT-3 or BERT.
- **External Knowledge Utilization**: RAG explicitly leverages external knowledge, while other models primarily rely on internal language models.
- **Focus on Factual Accuracy**: RAG is specifically designed to improve the factual accuracy and consistency of generated text, which may not be the primary focus of other generative models.

## Conclusion

RAG represents a significant advancement in generative AI, offering a hybrid approach that combines the strengths of retrieval-based and generative methods. By grounding the generation process in real-world knowledge, RAG enables the creation of more accurate, relevant, and coherent text. Despite certain challenges, RAG holds great promise for a wide range of NLP applications, including question answering, content creation, and text summarization. As research and development continue, RAG is poised to play an increasingly important role in the future of generative AI.